In [4]:
!pip install altair_saver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.5/844.5 kB 2.5 MB/s eta 0:00:002.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 3.2 MB/s eta 0:00:003.3 MB/s eta 0:00:01
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.35.37 which is incompatible.
sagemaker 2.232.2 requires attrs<24,>=23.1.0, but you have attrs 24.3.0 which is incompatible.


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
# from gensim import corpora, models
# from gensim.models.ldamodel import LdaModel
from nltk.tokenize import word_tokenize
import altair as alt
from altair_saver import save
import os
import re
from nltk.util import ngrams
from itertools import chain

import numpy as np
import warnings
warnings.simplefilter("ignore")

In [8]:
df = pd.read_csv('Data_mesh_publications_cleaned.csv')
df_sample = df. copy()
df

,Filename,Content,Publisher
0,Finding Your Way Through the Jungle of Big Dat...,Abstract—This paper presents a systematic revi...,IEEE
1,Enterprise Data Strategy A Decentralized Data ...,Abstract— As the enterprises experience expone...,IEEE
2,A Semantic Approach to Identifier Management i...,Semantic Approach to Identiﬁer Management inE...,IEEE
3,Towards Automating Federated Data Governance_c...,utomating Federated Data Governance‬‭Arif Wide...,IEEE
4,Breaking Down Data Silos Data Mesh to Achieve ...,Abstract— Data localization laws...,IEEE
5,A novel design for Data Processing Framework o...,novel design for Data Processing Framework of...,IEEE
6,Decentralized Data Governance as Part of a Dat...,Abstract —Data mesh is a socio-technical ...,IEEE
7,Privacy Engineering in the Data Mesh Towards a...,". Privacy engineering, emphasizing data protec...",Springer
8,Data Mesh as Distributed Data Platform for Lar...,. Rapid increase in data volume during the las...,Springer
9,ProMoTe A Data Product Model Template for Data...,. As the shortcomings of monolithic data platf...,Springer


# TFIDF

In [10]:
# Define a function to preprocess text data
def preprocess_text(text):
    # Remove non-alphanumeric characters and special characters
    text = re.sub(r'[^a-zA-Z0-9\s-]', '', text)

    return text

# Preprocess the PDF content before fitting it to the TF-IDF vectorizer
df['Content'] = df['Content'].apply(preprocess_text)

In [12]:
# def find_top_words_for_each_document(df, num_top_words=5):

#     # Create a TF-IDF vectorizer
#     tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(2, 4), token_pattern=r'\b\w[\w-]+\b')

#     # Fit and transform the PDF content
#     tfidf_matrix = tfidf_vectorizer.fit_transform(df['Content'])

#     # Get feature names (words) from the TF-IDF vectorizer
#     feature_names = tfidf_vectorizer.get_feature_names_out()

#     # Create a DataFrame with TF-IDF values
#     tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

#     # Now, for each document, find the top words based on TF-IDF scores
#     top_words_for_each_document = []
#     for i, row in enumerate(tfidf_df.iterrows()):
#         _, document_tfidf_scores = row
#         top_words_index = document_tfidf_scores.argsort()[-num_top_words:][::-1]
#         top_words = [(feature_names[index], document_tfidf_scores[index]) for index in top_words_index]
#         top_words_for_each_document.append(top_words)

#     # Create a new DataFrame with 'pdf_content' and 'top_words'
#     result_df = pd.DataFrame({'Filename': df['Filename'],'Year': df['Year'],'Publisher': df['Publisher'],'Publication_type': df['Publication_type'], 'Content': df['Content'], 'top_words': top_words_for_each_document})

#     return result_df

In [29]:
def find_top_words_for_each_document(df, num_top_words=5):
    df['Content'] = df['Content'].apply(preprocess_text)

    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(2, 4), token_pattern=r'\b\w[\w-]+\b')
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['Content'])
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Correcting split words in feature names
    #corrected_features = {name: name.replace('arti cial', 'artificial') for name in feature_names}
    #corrected_feature_names = [corrected_features[name] for name in feature_names]

    # Create a DataFrame with corrected feature names and TF-IDF values
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

    # Now, for each document, find the top words based on corrected TF-IDF scores
    top_words_for_each_document = []
    for i, row in tfidf_df.iterrows():
        top_words_index = row.argsort()[-num_top_words:][::-1]
        top_words = [(feature_names[index], row[index]) for index in top_words_index]
        top_words_for_each_document.append(top_words)

    # Create a new DataFrame with results
    result_df = pd.DataFrame({
        'Filename': df['Filename'],
        'Publisher': df['Publisher'],
        'Content': df['Content'],
        'top_words': top_words_for_each_document
    })

    return result_df


In [31]:
# Call the function
result_df_tfidf = find_top_words_for_each_document(df)
#result_df_tfidf

In [32]:
result_df_tfidf

,Filename,Publisher,Content,top_words
0,Finding Your Way Through the Jungle of Big Dat...,IEEE,AbstractThis paper presents a systematic revie...,"[(logical data, 0.11641801996090959), (data fa..."
1,Enterprise Data Strategy A Decentralized Data ...,IEEE,Abstract As the enterprises experience exponen...,"[(data mesh, 0.08969923415005512), (242023 141..."
2,A Semantic Approach to Identifier Management i...,IEEE,Semantic Approach to Identier Management inEn...,"[(informaticsuniversity oslooslo, 0.0557735721..."
3,Towards Automating Federated Data Governance_c...,IEEE,utomating Federated Data GovernanceArif Wider1...,"[(december 012024 111941, 0.08787187490507156)..."
4,Breaking Down Data Silos Data Mesh to Achieve ...,IEEE,Abstract Data localization laws ...,"[(data localization, 0.60319305234105), (data ..."
5,A novel design for Data Processing Framework o...,IEEE,novel design for Data Processing Framework of...,"[(type power, 0.11124776448664817), (new type ..."
6,Decentralized Data Governance as Part of a Dat...,IEEE,Abstract Data mesh is a socio-technical ...,"[(data mesh, 0.20996304939314842), (data mesh ..."
7,Privacy Engineering in the Data Mesh Towards a...,Springer,Privacy engineering emphasizing data protecti...,"[(data mesh, 0.11925977578619289), (data priva..."
8,Data Mesh as Distributed Data Platform for Lar...,Springer,Rapid increase in data volume during the last...,"[(process data, 0.06971585397358099), (distrib..."
9,ProMoTe A Data Product Model Template for Data...,Springer,As the shortcomings of monolithic data platfo...,"[(data products, 0.16727916842206844), (descri..."


# Results with filename

In [35]:
result_data_tfidf_filename = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Filename = row['Filename']
    top_words = row['top_words']

    # Iterate through each tuple in the list of top_words
    for word, tfidf in top_words:
        result_data_tfidf_filename.append({'Filename': Filename, 'topic_word': word, 'tfidf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_filename = pd.DataFrame(result_data_tfidf_filename)

# result_df_tfidf_filename

In [37]:
import pandas as pd

# Assuming result_df_3 is your DataFrame
# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', None) 
# pd.set_option('display.max_colwidth', None)# Show all rows

# Display the entire DataFrame
result_df_tfidf_filename


,Filename,topic_word,tfidf_value
0,Finding Your Way Through the Jungle of Big Dat...,logical data,0.116418
1,Finding Your Way Through the Jungle of Big Dat...,data fabric,0.111055
2,Finding Your Way Through the Jungle of Big Dat...,logical data warehouse,0.097027
3,Finding Your Way Through the Jungle of Big Dat...,data warehouse,0.076252
4,Finding Your Way Through the Jungle of Big Dat...,architecture paradigms,0.062082
...,...,...,...
255,Implementing Federated Governance in Data Mesh...,internet 2024 16,0.118921
256,Implementing Federated Governance in Data Mesh...,2024 16,0.118921
257,Implementing Federated Governance in Data Mesh...,internet 2024,0.118921
258,Implementing Federated Governance in Data Mesh...,data mesh,0.070161


In [39]:
result_df_tfidf_filename.iloc[49,1]

'data product model template'

In [41]:
result_df_tfidf_filename = result_df_tfidf_filename.replace('arti cial intelligence','artificial intelligence')
result_df_tfidf_filename = result_df_tfidf_filename.replace('quasi identi','quasi identifier')
result_df_tfidf_filename = result_df_tfidf_filename.replace('quasi identi er','quasi identifier')

result_df_tfidf_filename[result_df_tfidf_filename['topic_word'].str.contains('intelligence')]

,Filename,topic_word,tfidf_value


In [43]:
# Select the top terms for each PDF
top_terms_df_tfidf = result_df_tfidf_filename.groupby('Filename').head(5)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_tfidf['tfidf_value'] = top_terms_df_tfidf['tfidf_value'] + np.random.rand(top_terms_df_tfidf.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_tfidf).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y=alt.Y('Filename:N', axis=alt.Axis(labelFontSize=15, titleFontSize=15)),
    color=alt.Color('tfidf_value:Q', scale=alt.Scale(scheme='viridis'), legend=alt.Legend(labelFontSize=20,titleFontSize=25)),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tfidf_value", order="descending")],
    groupby=["Filename"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tfidf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle', fontWeight= 'normal', fontSize=15).encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.tfidf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_tfidf = (heatmap + text).properties(width=1100, height= 510)
chart_tfidf

alt.LayerChart(...)

In [45]:
save(chart_tfidf, r'vis\final_vis\tfidf_heatmap_all.html')

# Result with publisher

In [48]:
result_data_tfidf_publisher = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Publisher = row['Publisher']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_tfidf_publisher.append({'Publisher': Publisher, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_publisher = pd.DataFrame(result_data_tfidf_publisher)


result_df_tfidf_publisher

,Publisher,top_word,tf_idf_value
0,IEEE,logical data,0.116418
1,IEEE,data fabric,0.111055
2,IEEE,logical data warehouse,0.097027
3,IEEE,data warehouse,0.076252
4,IEEE,architecture paradigms,0.062082
...,...,...,...
255,miscellaneous,internet 2024 16,0.118921
256,miscellaneous,2024 16,0.118921
257,miscellaneous,internet 2024,0.118921
258,miscellaneous,data mesh,0.070161


In [49]:
result_df_tfidf_publisher = result_df_tfidf_publisher.replace('arti cial intelligence','artificial intelligence')
result_df_tfidf_publisher = result_df_tfidf_publisher.replace('quasi identi','quasi identifier')
result_df_tfidf_publisher = result_df_tfidf_publisher.replace('quasi identi er','quasi identifier')

result_df_tfidf_publisher[result_df_tfidf_publisher['top_word'].str.contains('identifier')]

,Publisher,top_word,tf_idf_value


In [52]:
# Select the top terms for each PDF
top_terms_df_publisher = result_df_tfidf_publisher.groupby('Publisher').head(5)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publisher['tf_idf_value'] = top_terms_df_publisher['tf_idf_value'] + np.random.rand(top_terms_df_publisher.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publisher).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y=alt.Y('Publisher:N', axis=alt.Axis(labelFontSize=15, titleFontSize=15)),
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis'), legend=alt.Legend(labelFontSize=20,titleFontSize=25)),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Publisher"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle',fontWeight='normal', fontSize=15).encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_publisher = (heatmap + text).properties(width=1200, height=150)
chart_publisher

alt.LayerChart(...)

In [54]:
# save(chart_tfidf_publisher_2, 'tfidf_heatmap_publisher.html')

# Results with Publisher

In [57]:
result_data_tfidf_publication_type = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Publication_type = row['Publication_type']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_tfidf_publication_type.append({'Publication_type': Publication_type, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_publication_type = pd.DataFrame(result_data_tfidf_publication_type)


result_df_tfidf_publication_type

KeyError: 'Publication_type'

In [ ]:
result_df_tfidf_publication_type = result_df_tfidf_publication_type.replace('arti cial intelligence','artificial intelligence')

result_df_tfidf_publication_type[result_df_tfidf_publication_type['top_word'].str.contains('intelligence')]

In [60]:
# Select the top terms for each PDF
top_terms_df_publication_type = result_df_tfidf_publication_type.groupby('Publication_type').head(5)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publication_type['tf_idf_value'] = top_terms_df_publication_type['tf_idf_value'] + np.random.rand(top_terms_df_publication_type.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publication_type).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y=alt.Y('Publication_type:N', axis=alt.Axis(labelFontSize=15, titleFontSize=15)),
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis'), legend=alt.Legend(labelFontSize=20,titleFontSize=25)),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Publication_type"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle',fontWeight='normal',fontSize=15).encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_publication_type = (heatmap + text).properties(width=1200, height=150)
chart_publication_type

NameError: name 'result_df_tfidf_publication_type' is not defined

In [62]:
result_data_tfidf_date = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Year = row['Year']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_tfidf_date.append({'Year': Year, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_data_tfidf_date = pd.DataFrame(result_data_tfidf_date)


result_data_tfidf_date

KeyError: 'Year'

In [41]:
# Select the top terms for each PDF
top_terms_df_tfidf_dates = result_data_tfidf_date.groupby('Year').head(5)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_tfidf_dates['tf_idf_value'] = top_terms_df_tfidf_dates['tf_idf_value'] + np.random.rand(top_terms_df_tfidf_dates.shape[0]) * 0.0001

# Create a base chart
# Create a base chart
base = alt.Chart(top_terms_df_tfidf_dates).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y=alt.Y('Year:N', axis=alt.Axis(labelFontSize=20, titleFontSize=20)),
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis'), legend=alt.Legend(labelFontSize=20,titleFontSize=25)),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Year"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle',fontWeight='normal',fontSize=20).encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_date_tfidf_type = (heatmap + text).properties(width=1300,height=100)
chart_date_tfidf_type

alt.LayerChart(...)